In [28]:
pip install scikit-learn


[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [40]:
#import necessary libraries
import pandas as pd
import numpy as np
from datetime import datetime, timedelta, time
import os
from sklearn.preprocessing import OneHotEncoder

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import pandas as pd
import numpy as np
import joblib





In [30]:
# Path to your CSV file
csv_path = "../data/cleaned_tasks.csv"

# Read the CSV into a DataFrame
df = pd.read_csv(csv_path)

# Preview the first 10 rows
df.head(10)


,task_id,team,role,user_id,task_name,description,priority,expected_duration,created_at,completed_at,completion_time,duration_error
0,7908,Engineering,Data Engineer,133,Performance profiling,Performance profiling for engineering team by ...,Low,4.0,2024-07-01 09:06:00,2024-07-01 11:54:33.263952,2.81,-1.19
1,2251,Operations,Logistics Coordinator,3,Resource planning,Resource planning for operations team by logis...,Medium,8.0,2024-07-01 09:18:00,2024-07-02 12:54:09.497882,27.60,19.60
2,2178,Marketing,Social Media Manager,43,Keyword research,Keyword research for marketing team by social ...,Low,7.0,2024-07-01 09:18:00,2024-07-02 10:22:49.919535,25.08,18.08
3,3587,Engineering,Backend Engineer,133,Integration testing,Integration testing for engineering team by ba...,Low,11.0,2024-07-01 09:19:00,2024-07-02 13:42:56.019706,28.40,17.40
4,8602,Engineering,QA Engineer,108,Implement new endpoint,Implement new endpoint for engineering team by...,Low,11.0,2024-07-01 09:23:00,2024-07-02 16:31:21.089017,31.14,20.14
5,7164,Engineering,Backend Engineer,125,Fix backend API bug,Fix backend API bug for engineering team by ba...,High,5.0,2024-07-01 09:25:00,2024-07-02 09:24:34.687681,23.99,18.99
6,2092,Operations,Logistics Coordinator,159,Process documentation,Process documentation for operations team by l...,Low,6.0,2024-07-01 09:28:00,2024-07-02 09:50:09.882155,24.37,18.37
7,9892,Engineering,Data Engineer,77,Write unit tests,Write unit tests for engineering team by data ...,Low,7.0,2024-07-01 09:51:00,2024-07-04 10:10:12.149629,72.32,65.32
8,9008,Engineering,Frontend Engineer,164,Database schema update,Database schema update for engineering team by...,Medium,4.0,2024-07-01 09:52:00,2024-07-01 13:09:16.261225,3.29,-0.71
9,3112,Engineering,Backend Engineer,98,Performance profiling,Performance profiling for engineering team by ...,Low,9.0,2024-07-01 09:55:00,2024-07-02 12:10:21.833520,26.26,17.26


## Team Feature Engineering

**Rationale:**  
Teams may work at different speeds and have varying workloads. Capturing team-related statistics helps the model understand baseline performance and potential delays.  
We create the following features:  
1. **One-hot encoding of team** → allows the model to distinguish teams categorically.  
2. **Team average completion** → captures the historical average completion time per team.  
3. **Team backlog** → counts incomplete tasks per team to reflect workload.  
4. **Team completion standard deviation** → captures variability in team performance.


In [31]:
# 1. One-hot encode 'team' for categorical distinction
df = pd.get_dummies(df, columns=['team'], prefix='team', drop_first=True)

# 2. Derive task status: 'completed' or 'incomplete'
df['status'] = df['completed_at'].apply(lambda x: 'completed' if pd.notnull(x) else 'incomplete')

# 3. Function to add average completion time per team (train/test safe)
def add_team_avg_completion(X_train, X_test, y_train):
    # Compute team-level average completion using training target
    team_avg = y_train.groupby(X_train['team']).mean()
    # Map to training and test sets; fill missing values in test with overall mean
    X_train['team_avg_completion'] = X_train['team'].map(team_avg)
    X_test['team_avg_completion'] = X_test['team'].map(team_avg).fillna(y_train.mean())
    return X_train, X_test

# 4. Function to compute team backlog (# of incomplete tasks)
def add_team_backlog(X):
    backlog = X[X['status'] != 'completed'].groupby('team').size()
    X['team_backlog'] = X['team'].map(backlog).fillna(0)
    return X

# 5. Function to compute team completion variability (standard deviation)
def add_team_completion_std(X_train, X_test, y_train):
    team_std = y_train.groupby(X_train['team']).std()
    X_train['team_completion_std'] = X_train['team'].map(team_std)
    X_test['team_completion_std'] = X_test['team'].map(team_std).fillna(y_train.std())
    return X_train, X_test


## Role Feature Engineering

**Rationale:**  
Different roles may have varying completion speeds and workload patterns.  
Capturing role-based statistics allows the model to account for these differences.  
We create the following features:  
1. **One-hot encoding of role** → categorical distinction for linear models.  
2. **Role average completion** → historical average completion time per role.  
3. **Role completion standard deviation** → variability of completion time per role.  
4. **Role backlog** → number of incomplete tasks per role.  
5. **Role task count** → total number of tasks per role, reflecting workload.


In [32]:
# 1. Derive task status early
df['status'] = df['completed_at'].apply(lambda x: 'completed' if pd.notnull(x) else 'incomplete')

# 2. Total task count per role (reflect workload)
role_task_count = df.groupby('role').size()
df['role_task_count'] = df['role'].map(role_task_count)

# 3. Role backlog (# incomplete tasks)
backlog = df[df['status'] != 'completed'].groupby('role').size()
df['role_backlog'] = df['role'].map(backlog).fillna(0)

# 4. Functions for train/test (these now work because 'role' is still present)
def add_role_avg_completion(X_train, X_test, y_train):
    role_avg = y_train.groupby(X_train['role']).mean()
    X_train['role_avg_completion'] = X_train['role'].map(role_avg)
    X_test['role_avg_completion'] = X_test['role'].map(role_avg).fillna(y_train.mean())
    return X_train, X_test

def add_role_completion_std(X_train, X_test, y_train):
    role_std = y_train.groupby(X_train['role']).std()
    X_train['role_completion_std'] = X_train['role'].map(role_std)
    X_test['role_completion_std'] = X_test['role'].map(role_std).fillna(y_train.std())
    return X_train, X_test

# 5. LAST STEP: One-hot encode 'role'
df = pd.get_dummies(df, columns=['role'], drop_first=True)


## User Feature Engineering

Rationale  
Users have different productivity, reliability, and work patterns. Capturing user-level statistics allows the model to account for personal efficiency, workload, and behavior.  

Features created  
1. Workload features: total tasks assigned, total tasks completed, completion rate  
2. Efficiency features: average, median, and standard deviation of completion time, expected duration, and duration error; on-time rate  
3. Priority performance: high-priority task completion, average completion time and error for high-priority tasks  
4. Behavioral features: average start and completion hour, weekend activity rates  
5. Reliability features: task dropout rate, performance variability  
6. Experience / tenure: active days, experience level


In [33]:
# -----------------------------
# USER-LEVEL FEATURE ENGINEERING
# -----------------------------

# Ensure datetime columns are parsed correctly
df['created_at'] = pd.to_datetime(df['created_at'], errors='coerce')
df['completed_at'] = pd.to_datetime(df['completed_at'], errors='coerce')

# -----------------------------
# 1. Helper Time Features
# -----------------------------
df['created_date'] = df['created_at'].dt.date
df['completed_date'] = df['completed_at'].dt.date
df['created_hour'] = df['created_at'].dt.hour
df['completed_hour'] = df['completed_at'].dt.hour
df['created_weekday'] = df['created_at'].dt.weekday
df['completed_weekday'] = df['completed_at'].dt.weekday
df['created_weekend'] = df['created_weekday'].isin([5, 6]).astype(int)
df['completed_weekend'] = df['completed_weekday'].isin([5, 6]).astype(int)
df['is_high_priority'] = df['priority'].str.lower().isin(['high', 'urgent']).astype(int)

# -----------------------------
# 2. Workload Features
# -----------------------------
user_groups = df.groupby('user_id')
workload = user_groups.agg(
    total_tasks_assigned=('task_name', 'count'),
    total_tasks_completed=('completed_at', lambda x: x.notna().sum()),
)
workload['completion_rate'] = (
    workload['total_tasks_completed'] / workload['total_tasks_assigned']
)

# -----------------------------
# 3. Efficiency Features
# -----------------------------
efficiency = user_groups.agg(
    avg_completion_time=('completion_time', 'mean'),
    median_completion_time=('completion_time', 'median'),
    std_completion_time=('completion_time', 'std'),
    avg_expected_duration=('expected_duration', 'mean'),
    avg_duration_error=('duration_error', 'mean'),
    median_duration_error=('duration_error', 'median'),
    std_duration_error=('duration_error', 'std'),
)
df['on_time'] = (df['completion_time'] <= df['expected_duration']).astype(int)
efficiency['on_time_rate'] = user_groups['on_time'].mean()

# -----------------------------
# 4. Priority Performance Features
# -----------------------------
priority_perf = user_groups.agg(
    high_priority_task_count=('is_high_priority', 'sum'),
    high_priority_on_time_rate=('on_time', lambda x: x[df['is_high_priority'] == 1].mean()),
)

high_priority = df[df['is_high_priority'] == 1]
hp_group = high_priority.groupby('user_id')
priority_perf['avg_completion_time_high_priority'] = hp_group['completion_time'].mean()
priority_perf['avg_error_high_priority'] = hp_group['duration_error'].mean()

# -----------------------------
# 5. Behavioral Features
# -----------------------------
behavior = user_groups.agg(
    avg_start_hour=('created_hour', 'mean'),
    avg_completion_hour=('completed_hour', 'mean'),
    weekend_creation_rate=('created_weekend', 'mean'),
    weekend_completion_rate=('completed_weekend', 'mean'),
)

# -----------------------------
# 6. Reliability Features
# -----------------------------
reliability = user_groups.agg(
    task_dropout_rate=('completed_at', lambda x: x.isna().mean()),
    performance_variance=('duration_error', 'std'),
)

# -----------------------------
# 7. Experience / Tenure Features
# -----------------------------
experience = user_groups.agg(
    first_task_date=('created_date', 'min'),
    last_task_date=('created_date', 'max'),
)

# Fix: ensure timedelta is consistent
experience['active_days'] = pd.to_timedelta(
    experience['last_task_date'] - experience['first_task_date']
).dt.days.replace(0, 1)

experience['experience_level'] = (
    workload['total_tasks_assigned'] / experience['active_days']
)

# -----------------------------
# 8. Merge All User-Level Features
# -----------------------------
user_features = (
    workload
    .join(efficiency)
    .join(priority_perf)
    .join(behavior)
    .join(reliability)
    .join(experience)
)

user_features = user_features.fillna(0)
user_features.reset_index(inplace=True)

user_features.head()


,user_id,total_tasks_assigned,total_tasks_completed,completion_rate,avg_completion_time,median_completion_time,std_completion_time,avg_expected_duration,avg_duration_error,median_duration_error,...,avg_start_hour,avg_completion_hour,weekend_creation_rate,weekend_completion_rate,task_dropout_rate,performance_variance,first_task_date,last_task_date,active_days,experience_level
0,1,52,52,1.0,28.772692,21.595,28.711934,5.057692,23.715000,16.91,...,12.538462,12.634615,0.0,0.0,0.0,27.355573,2024-07-03,2024-12-31,181,0.287293
1,2,57,57,1.0,37.963333,25.230,35.968715,6.017544,31.945789,19.07,...,12.701754,12.807018,0.0,0.0,0.0,34.266113,2024-07-01,2024-12-27,179,0.318436
2,3,52,52,1.0,29.838846,23.310,27.236014,5.692308,24.146538,17.35,...,12.307692,13.057692,0.0,0.0,0.0,25.726402,2024-07-01,2024-12-25,177,0.293785
3,4,53,53,1.0,39.564340,24.790,32.007421,5.566038,33.998302,19.43,...,12.509434,12.132075,0.0,0.0,0.0,30.726003,2024-07-01,2024-12-31,183,0.289617
4,5,58,58,1.0,36.862759,25.930,31.326550,6.310345,30.552414,18.83,...,12.362069,12.379310,0.0,0.0,0.0,29.711336,2024-07-08,2024-12-25,170,0.341176


## Task Name and Priority Feature Engineering

Rationale  
Tasks can vary in complexity and expected effort. Capturing task-specific and priority information helps the model adjust predictions based on historical performance.  

Features created  
1. Task average completion: historical average completion time per task  
2. Task description: dropped because task_name already captures the task  
3. Priority encoded: numerical encoding of Low, Medium, High for linear models or numerical comparisons


In [34]:
# Task-level feature: average completion time per task
task_avg_completion = df.groupby('task_name')['completion_time'].mean()
df['task_avg_completion'] = df['task_name'].map(task_avg_completion)

# Drop description since task_name captures the task
df = df.drop(columns=['description'])

# Encode priority for linear models
priority_mapping = {'Low': 0, 'Medium': 1, 'High': 2}
df['priority_encoded'] = df['priority'].map(priority_mapping)


## Task Name and Priority Feature Engineering

**Rationale**  
Tasks can vary in complexity and expected effort. Capturing **task-specific** and **priority information** helps the model adjust predictions based on historical performance.  

**Features created**  
1. **Task average completion**: historical average completion time per task  
2. **Task description**: dropped because **task_name** already captures the task  
3. **Priority encoded**: numerical encoding of **Low, Medium, High** for linear models or numerical comparisons


In [35]:
# Task-level feature: average completion time per task
task_avg_completion = df.groupby('task_name')['completion_time'].mean()
df['task_avg_completion'] = df['task_name'].map(task_avg_completion)

# Drop description if it exists
df = df.drop(columns=['description'], errors='ignore')

# Encode priority for linear models
priority_mapping = {'Low': 0, 'Medium': 1, 'High': 2}
df['priority_encoded'] = df['priority'].map(priority_mapping)


## Temporal Feature Engineering

**Rationale**  
Task start times can influence completion time due to day-of-week patterns, hourly workload variations, or weekend effects. Capturing temporal features helps the model adjust predictions based on when a task was created.  

**Features created**  
1. **Start day of week**: numeric (0 = Monday, 6 = Sunday)  
2. **Start hour**: hour of task creation  
3. **Weekend flag**: 1 if task created on Saturday or Sunday, 0 otherwise  
4. **Expected duration**: kept as numeric for modeling  
5. **Completion ratio (optional)**: completion_time / expected_duration


In [36]:
# Drop completed_at since we already have completion_time
df = df.drop(columns=['completed_at'])

# Keep expected_duration as numeric
# Optional normalized feature
# df['completion_ratio'] = df['completion_time'] / df['expected_duration']

# Ensure created_at is datetime
df['created_at'] = pd.to_datetime(df['created_at'])

# Day of week (0 = Monday)
df['start_dayofweek'] = df['created_at'].dt.dayofweek

# Hour of day
df['start_hour'] = df['created_at'].dt.hour

# Weekend flag
df['is_weekend'] = df['start_dayofweek'].isin([5,6]).astype(int)

# Optional: drop created_at if only using engineered features
# df = df.drop(columns=['created_at'])

# Display the engineered temporal features
print(df[['expected_duration', 'start_dayofweek', 'start_hour', 'is_weekend']].head())

#droping Task ID column as it is not needed for modeling
df.drop(columns="task_id", inplace=True)


   expected_duration  start_dayofweek  start_hour  is_weekend
0                4.0                0           9           0
1                8.0                0           9           0
2                7.0                0           9           0
3               11.0                0           9           0
4               11.0                0           9           0


,user_id,task_name,priority,expected_duration,created_at,completion_time,duration_error,team_Engineering,team_Finance,team_Marketing,...,completed_weekday,created_weekend,completed_weekend,is_high_priority,on_time,task_avg_completion,priority_encoded,start_dayofweek,start_hour,is_weekend
0,133,Performance profiling,Low,4.0,2024-07-01 09:06:00,2.81,-1.19,True,False,False,...,0,0,0,0,1,41.069146,0,0,9,0
1,3,Resource planning,Medium,8.0,2024-07-01 09:18:00,27.60,19.60,False,False,False,...,1,0,0,0,0,28.071157,1,0,9,0
2,43,Keyword research,Low,7.0,2024-07-01 09:18:00,25.08,18.08,False,False,True,...,1,0,0,0,0,36.815475,0,0,9,0
3,133,Integration testing,Low,11.0,2024-07-01 09:19:00,28.40,17.40,True,False,False,...,1,0,0,0,0,43.910037,0,0,9,0
4,108,Implement new endpoint,Low,11.0,2024-07-01 09:23:00,31.14,20.14,True,False,False,...,1,0,0,0,0,42.704972,0,0,9,0
5,125,Fix backend API bug,High,5.0,2024-07-01 09:25:00,23.99,18.99,True,False,False,...,1,0,0,1,0,33.908055,2,0,9,0
6,159,Process documentation,Low,6.0,2024-07-01 09:28:00,24.37,18.37,False,False,False,...,1,0,0,0,0,25.857906,0,0,9,0
7,77,Write unit tests,Low,7.0,2024-07-01 09:51:00,72.32,65.32,True,False,False,...,3,0,0,0,0,40.415691,0,0,9,0
8,164,Database schema update,Medium,4.0,2024-07-01 09:52:00,3.29,-0.71,True,False,False,...,0,0,0,0,1,37.233168,1,0,9,0
9,98,Performance profiling,Low,9.0,2024-07-01 09:55:00,26.26,17.26,True,False,False,...,1,0,0,0,0,41.069146,0,0,9,0


In [42]:
class TaskFeatureEngineer(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.task_avg_completion_ = None
        self.priority_mapping = {'Low': 0, 'Medium': 1, 'High': 2}

    def fit(self, X, y=None):
        self.task_avg_completion_ = y.groupby(X['task_name']).mean()
        return self

    def transform(self, X):
        X = X.copy()
        X['task_avg_completion'] = X['task_name'].map(self.task_avg_completion_).fillna(0)
        X['priority_encoded'] = X['priority'].map(self.priority_mapping)
        X = X.drop(columns=['description'], errors='ignore')
        X = X.drop(columns=['task_id'], errors='ignore')
        return X

class TemporalFeatures(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X = X.copy()
        X['created_at'] = pd.to_datetime(X['created_at'])
        X['start_dayofweek'] = X['created_at'].dt.dayofweek
        X['start_hour'] = X['created_at'].dt.hour
        X['is_weekend'] = X['start_dayofweek'].isin([5,6]).astype(int)
        return X

class RoleStats(BaseEstimator, TransformerMixin):
    def fit(self, X, y):
        df = X.copy()
        df['target'] = y
        self.role_avg_ = df.groupby('role')['target'].mean()
        self.role_std_ = df.groupby('role')['target'].std()
        return self

    def transform(self, X):
        X = X.copy()
        X['role_avg_completion'] = X['role'].map(self.role_avg_).fillna(self.role_avg_.mean())
        X['role_completion_std'] = X['role'].map(self.role_std_).fillna(self.role_std_.mean())
        return X

def get_pipeline():
    from sklearn.pipeline import Pipeline
    from sklearn.compose import ColumnTransformer
    from sklearn.preprocessing import OneHotEncoder

    return Pipeline([
        ("task_features", TaskFeatureEngineer()),
        ("temporal_features", TemporalFeatures()),
        ("role_features", RoleStats()),
        ("encode_role",
         ColumnTransformer([
             ("onehot", OneHotEncoder(handle_unknown="ignore"), ["role"])
         ], remainder="passthrough")
        )
    ])

In [43]:
pipeline = get_pipeline()  # unfitted
joblib.dump(pipeline, "feature_pipeline_unfitted.pkl")

['feature_pipeline_unfitted.pkl']